In [ ]:
# coding: utf-8
import numpy as np
from matplotlib.pyplot import subplots, show
from kapteyn import kmpfit

In [ ]:
def model(p,x):
    return p[0]*np.exp(-(x-p[2])**2/p[1]**2)+p[3]

def model(p,x):
    return p[0]*np.exp(-np.abs(x-p[2])/p[1])+p[3]

def model(p,x):
    return p[0]*p[1]**2/((x-p[2])**2 + p[1]**2)+p[3]

fitobj=kmpfit.simplefit(model,[6e-10, 1e-6, 0.55795, 0.4e-10], period, chisq)

In [ ]:
fitobj.params

In [ ]:
fig, ax = subplots()
data = np.load("chisqs.npz")
period = data['period']
chisq = data['chisq']
ax.plot(period,chisq)
ax.plot(period, model(fitobj.params, period))
#ax.set(xlim=(0.033745,0.033748))
#ax.set_xlim(56.5,57)
#ax.axvline(dbperiod)
show()

In [ ]:
from astropy.io import fits
import numpy as np

hdulist = fits.open("data/obs-10-04-2018/B2016+28_10-04-2018.fits")
col_period = fits.Column(name='period', array=data['period'], format='D')
col_DM = fits.Column(name='DM', array=data['DM'], format='D')
col_chisq = fits.Column(name='chisq', array=data['chisq'], format='D')
t = fits.BinTableHDU.from_columns([col_period, col_DM, col_chisq])
t.header['fitpars'] = str(fitobj.params)
t.header['bestp'] = fitobj.params[2]
t.header['fitfunc'] = "Lorentzian"
hdulist.append(t)
hdulist.writeto("data/obs-10-04-2018/B2016+28_10-04-2018-withP.fits")